# Hướng dẫn truy vấn dữ liệu thị giác dùng fiftyone

Đây là hướng dẫn dùng cho các đội tham dự AI Challenge 2023. Hướng dẫn này nhằm mục đích giới thiệu cho các đội một phương pháp cơ bản để truy vấn dữ liệu dựa trên thông tin BTC cung cấp và giới thiệu công cụ fiftyone để hỗ trợ đội thi đánh giá kết quả.

## Cài đặt ban đầu

Bạn cần cài đặt môi trường để chạy được notebook này trên máy tính cá nhân của bạn. Hướng dẫn này không bao gồm phần cài đặt môi trường. Khuyến nghị: các bạn có thể cài đặt [Anaconda](https://docs.anaconda.com/free/anaconda/install/windows/).

## Cài đặt các thư viện FiftyOne và PyTorch
Hướng dẫn này dùng fiftyone là công cụ để trực quan dữ liệu và pytorch là backend chính cho các thuật toán máy học.

### Lưu ý: Đối với các bạn dùng Windows nên dùng bản fiftyone **v0.21.4**, không nên dùng bản mới nhất!

In [1]:
! pip install fiftyone==0.21.4
! pip install torch torchvision torchaudio
! pip install ipywidgets
! pip install lancedb
! pip install httpcore==0.17.3
! pip install httpx==0.24.1
! pip install h11==0.14.0

Load dữ liệu keyframe từ thư mục chứa keyframe. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một Sample. Tất cả các Sample được lưu trong Dataset.

In [2]:
import fiftyone as fo
import fiftyone.brain as fob
import numpy as np
from glob import glob
import json
import os

Load dữ liệu keyframe từ thư mục chứa keyframe. Trong hướng dẫn này tất cả các file Keyframes_L*.zip được giải nén vào thư mục `D:\AIC\Keyframes`. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một `Sample`. Tất cả các `Sample` được lưu trong `Dataset`.

In [ ]:
# load dataset
# dataset = fo.Dataset.from_images_dir('/Users/duyennguyen/Downloads/AIC/Batch_01/Keyframes', name=None, tags=None, recursive=True)

 100% |█████████████| 87306/87306 [5.8s elapsed, 0s remaining, 15.2K samples/s]      


In [3]:
# Reload dataset

import fiftyone as fo

# The directory containing the dataset to import
dataset_dir = "./lancedb_index_051023"

# The type of the dataset being imported
dataset_type = fo.types.FiftyOneDataset  # for example

# Import the dataset
dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=dataset_type,
)

Importing samples...
 100% |█████████████| 87306/87306 [1.2s elapsed, 0s remaining, 73.9K samples/s]         
Import complete


In [ ]:
print(dataset.first())

<Sample: {
    'id': '651ea66118a59293a3c7c199',
    'media_type': 'image',
    'filepath': '/Users/duyennguyen/Downloads/AIC/Batch_01/lancedb_index_051023/data/0001.jpg',
    'tags': [],
    'metadata': None,
    'video': 'L07_V001',
    'frameid': '0001',
    'object_faster_rcnn': <Detections: {
        'detections': [
            <Detection: {
                'id': '651ea67c18a59293a3c916a3',
                'attributes': {},
                'tags': [],
                'label': 'Traffic sign',
                'bounding_box': [0.20671254, 0.2666586, 0.5882361, 0.4689785],
                'mask': None,
                'confidence': 0.8469662,
                'index': None,
            }>,
            <Detection: {
                'id': '651ea67c18a59293a3c916a4',
                'attributes': {},
                'tags': [],
                'label': 'Traffic sign',
                'bounding_box': [0.19411297, 0.28320366, 0.19830398, 0.3873986],
                'mask': None,
           

In [4]:
session = fo.launch_app(dataset, port =5152)
session.wait()


Notebook sessions cannot wait


Sau khi dữ liệu đã load lên xong. Bạn có thể truy cập vào đường vào ứng dụng web của fiftyone từ [http://localhost:5152](http://localhost:5152)

Hoặc bạn có thể chạy cell bên dưới để mở tab mới cho ứng dụng web fiftyone

### Trích xuất thêm thông tin tên của video và frameid
Thông tin `video` và `frameid` sẽ được lấy từ tên của tập tin keyframe.

In [ ]:
# for sample in dataset:
#     _, sample['video'], sample['frameid'] = sample['filepath'][:-4].rsplit('/', 2)
#     sample.save()

### Thêm thông tin kết quả của object detection.

Bước này có thể tốn của bạn nhiều thời gian để đọc hết tất cả các dữ liệu về object detection. Bạn có thể bỏ qua cell này và chạy cell này sau nếu muốn thử thêm các thông tin về vector CLIP embedding trước.

In [ ]:
# #label_set = set()

# for sample in dataset:
#     object_path = f"/Users/duyennguyen/Downloads/AIC/Batch_01/objects/{sample['video']}/{sample['frameid']}.json"
#     with open(object_path) as jsonfile:
#         det_data = json.load(jsonfile)
#     detections = []
#     for cls, box, score in zip(det_data['detection_class_entities'], det_data['detection_boxes'], det_data['detection_scores']):
#         # Convert to [top-left-x, top-left-y, width, height]
#         boxf = [float(box[1]), float(box[0]), float(box[3]) - float(box[1]), float(box[2]) - float(box[0])]
#         scoref = float(score)

#         # Only add objects with confidence > 0.4
#         if scoref > 0.4:
#             detections.append(
#                 fo.Detection(
#                     label=cls,
#                     bounding_box= boxf,
#                     confidence=float(score)
#                 )
#             )
#     # for detection in detections:
#     #     label_set.add(detection.label)
        
#     sample["object_faster_rcnn"] = fo.Detections(detections=detections)
#     sample.save()
    
# # # save label_set as a json file
# # with open('label_set.json', 'w') as f:
# #     json.dump(list(label_set), f)

### Thêm thông tin CLIP embedding.

In [ ]:
# all_keyframe = glob('/Users/duyennguyen/Downloads/AIC/Batch_01/Keyframes/*/keyframes-*/*/*.jpg')
# video_keyframe_dict = {}
# all_video = glob('/Users/duyennguyen/Downloads/AIC/Batch_01/Keyframes/*/keyframes-*/*')
# all_video = [v.rsplit('/',1)[-1] for v in all_video]

Đọc thông tin clip embedding được cung cấp.

Lưu ý: Các bạn cần tải đúng bản CLIP embedding từ model **CLIP ViT-B/32**

Tạo dictionary `video_keyframe_dict` với `video_keyframe_dict[video]` thông tin danh sách `keyframe` của `video`

In [ ]:
# for kf in all_keyframe:
#     _, vid, kf = kf[:-4].rsplit('/',2)
#     if vid not in video_keyframe_dict.keys():
#         video_keyframe_dict[vid] = [kf]
#     else:
#         video_keyframe_dict[vid].append(kf)

Do thông tin vector CLIP embedding được cung cấp được lưu theo từng video nhầm mục đích tối ưu thời gian đọc dữ liệu. Cần sort lại danh sách `keyframe` của từng `video` để đảm bảo thứ tự đọc đúng với vector embedding được cung cấp.

In [ ]:
# for k,v in video_keyframe_dict.items():
#     video_keyframe_dict[k] = sorted(v)

Tạo dictionary `embedding_dict` với `embedding_dict[video][keyframe]` lưu thông tin vector CLIP embedding của `keyframe` trong `video` tương ứng

In [ ]:
# embedding_dict = {}
# for v in all_video:
#     clip_path = f'/Users/duyennguyen/Downloads/AIC/Batch_01/clip-features-vit-b32/{v}.npy'
#     a = np.load(clip_path)
#     embedding_dict[v] = {}
#     for i,k in enumerate(video_keyframe_dict[v]):
#         embedding_dict[v][k] = a[i]

Tạo danh sách `clip_embedding` ứng với danh sách `sample` trong `dataset`.

In [ ]:
# clip_embeddings = []
# for sample in dataset:
#     clip_embedding = embedding_dict[sample['video']][sample['frameid']]
#     clip_embeddings.append(clip_embedding)


In [ ]:
# # Clean up    
# # Delete run record from FiftyOne
# dataset.delete_brain_run("lancedb_index") # milvus_index, img_sim, tmp

In [35]:
# lancedb_index = fob.compute_similarity(

#     # # default
#     # dataset,
#     # model="clip-vit-base32-torch",      # store model's name for future use
#     # embeddings=clip_embeddings,         # precomputed image embeddings
#     # backend="sklearn",                  
#     # brain_key="img_sim",                


#     # lancedb
#     dataset,
#     # embeddings=clip_embeddings,         # precomputed image embeddings
#     model="clip-vit-base32-torch",      # store model's name for future use
#     brain_key="lancedb_index",                # img_sim, tmp, milvus_index, lancedb_index
#     backend="lancedb",                  # sklearn, qdrant, pinecone, milvus, lancedb

#     # # milvus
#     # dataset,
#     # model="clip-vit-base32-torch",      # store model's name for future use
#     # # embeddings=clip_embeddings,         # precomputed image embeddings
#     # backend="lancedb",                  # sklearn, qdrant, pinecone, milvus, lancedb
#     # brain_key="lancedb_index",                # img_sim, tmp, milvus_index, lancedb_index
#     # # uri="tcp://localhost:19530",        # milvus server uri

#     # # qdrant
#     # dataset,
#     # model="clip-vit-base32-torch",      # store model's name for future use
#     # # embeddings=clip_embeddings,         # precomputed image embeddings
#     # backend="qdrant",                  # sklearn, qdrant, pinecone, milvus, lancedb
#     # brain_key="qdrant_index",                # img_sim, tmp, milvus_index, lancedb_index

#     # # pinecone
#     # dataset,
#     # model="clip-vit-base32-torch",      # store model's name for future use
#     # # embeddings=clip_embeddings,       # precomputed image embeddings
#     # backend="pinecone",                 # sklearn, qdrant, pinecone, milvus, lancedb
# # brain_key="pinecone_index",             # img_sim, tmp, milvus_index, lancedb_index
        
# )



Computing embeddings...
  21% |██|----------| 18533/87306 [17.5m elapsed, 1.2h remaining, 10.8 samples/s]   

In [17]:
print(dataset.get_brain_info('lancedb_index'))
lancedb_index = dataset.load_brain_results('lancedb_index')
print(lancedb_index.)

{
    "key": "lancedb_index",
    "version": "0.21.4",
    "timestamp": "2023-10-05T12:19:15.836000",
    "config": {
        "method": "lancedb",
        "cls": "fiftyone.brain.internal.core.lancedb.LanceDBSimilarityConfig",
        "embeddings_field": null,
        "model": "clip-vit-base32-torch",
        "patches_field": null,
        "supports_prompts": true,
        "table_name": "fiftyone-2023-10-05-19-04-48",
        "metric": "cosine"
    }
}
<bound method LanceDBSimilarityIndex.__init__ of <fiftyone.brain.internal.core.lancedb.LanceDBSimilarityIndex object at 0x2c5e59330>>


In [19]:
print(fob.brain_config)


{
    "default_similarity_backend": "sklearn",
    "similarity_backends": {
        "lancedb": {
            "config_cls": "fiftyone.brain.internal.core.lancedb.LanceDBSimilarityConfig"
        },
        "milvus": {
            "config_cls": "fiftyone.brain.internal.core.milvus.MilvusSimilarityConfig"
        },
        "pinecone": {
            "config_cls": "fiftyone.brain.internal.core.pinecone.PineconeSimilarityConfig"
        },
        "qdrant": {
            "config_cls": "fiftyone.brain.internal.core.qdrant.QdrantSimilarityConfig"
        },
        "sklearn": {
            "config_cls": "fiftyone.brain.internal.core.sklearn.SklearnSimilarityConfig"
        }
    }
}


In [15]:
import lancedb
db = lancedb.connect('./.lancedb')
db.create_table('lancedb_index', lancedb_index.table)

lancedb_table = db.open_table('lancedb_index')
query = dataset.first().id  # query by sample ID
query_results = lancedb_table.search(query).limit(10)


FileNotFoundError: Table None does not exist. Please first call db.create_table(None, data)

In [ ]:
# Step 4: Query your data
query = dataset.first().id  # query by sample ID
view = dataset.sort_by_similarity(
    query,
    brain_key="lancedb_index",
    k=10,  # limit to 10 most similar samples
)

# Step 5 (optional): Cleanup

# # Delete the LanceDB table
# lancedb_index.cleanup()

# # Delete run record from FiftyOne
# dataset.delete_brain_run("lancedb_index")

651ea66118a59293a3c7c199


AttributeError: 'NoneType' object has no attribute 'to_pandas'

In [ ]:
# Retrieve the raw LanceDB table
table = lancedb_index.table

df = table.to_pandas()  # get the table as a pandas dataframe
pa = table.to_arrow()   # get the table as an arrow table

df.head()
# # export the table to a csv file
# df.to_csv("./lancedb_index.csv")

AttributeError: 'NoneType' object has no attribute 'to_pandas'

In [ ]:
# dataset.list_brain_runs(type=fob.Similarity)

In [ ]:
dataset.export(
    './lancedb_index_051023',
    dataset_type=fo.types.FiftyOneDataset
    )

In [ ]:
# import fiftyone as fo

# # The directory containing the dataset to import
# dataset_dir = "./lancedb_index_051023"

# # The type of the dataset being imported
# dataset_type = fo.types.FiftyOneDataset  # for example

# # Import the dataset
# dataset = fo.Dataset.from_dir(
#     dataset_dir=dataset_dir,
#     dataset_type=dataset_type,
# )

In [ ]:
# dataset.load_brain_view("lancedb_index")

In [ ]:
# print(dataset.first()['object_faster_rcnn'])

In [ ]:
# # https://docs.voxel51.com/user_guide/brain.html#text-similarity

# # Query text
# query = "kites high in the air"

# ## Image similarity
# # Perform a text query
# view = dataset.sort_by_similarity(
#     query, 
#     k=15, 

#     # # default
#     # brain_key="img_sim"         

#     # lancedb
#     # brain_key="lancedb_index"

#     # # milvus
#     # brain_key="milvus_index"

#     # # qdrant
#     # brain_key="qdrant_index"

#     # # pinecone
#     # brain_key="pinecone_index"
# )

# session.view = view

## Từ đây các bạn có thể thử các tính năng search, filter trên ứng dụng fiftyone.

In [ ]:
# # Bạn cần phải cài version umap-learn hỗ trợ.
# fob.compute_visualization(
#     dataset,
#     embeddings=clip_embeddings,   # precomputed image embeddings
#     brain_key="img_viz",          # img_viz, tmp, gt_viz
#     method="umap",                # umap, tsne, pca
#     seed=51,                      # random seed
# )


In [ ]:
# from fiftyone import ViewField as F

# # Restrict to the 10 most common classes
# counts = dataset.count_values("ground_truth.detections.label")
# classes = sorted(counts, key=counts.get, reverse=True)[:10]
# view = dataset.filter_labels("ground_truth", F("label").is_in(classes))
# session.view = view